# Chatbot made with Python

This is my(Adheesh Trivedi's) implementation of chatbot made in python programming language and libraries such as `numpy`, `nltk` (Natural Language Tool-kit), `tensorflow` (The Giant Machine Learning library) and few other.
```asciidoc
Creator     :: Adheesh Trivedi
Class       :: 12th
```

### Importing Libraries

In [1]:
try:
    from nltk.stem.lancaster import LancasterStemmer
    from nltk import word_tokenize
    import numpy as np
    import pickle as pkl
    from json import load
    from typing import List, Tuple
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.optimizers import SGD
except ModuleNotFoundError as err:
    print("Module named " + err.name + " was not found in the path.")

### Defining some globals

* `stemmer` :\
    It's a `LancasterStemmer` instance that is gonna help us classify `word`, `words`, or `wording` as word, or simply say, will remove the tenses part or any preffix or suffix in a word that dosn't change the meaning of the word much.
* `intents` :\
    It's a list of dictionaries that is going to have chatbot's all `intents`, `patterns` (The list of patterns of statements that falls under that intent), and a list of responses in the following pattern:
    ```json
        {"intent": "farewell",
        "patterns": ["goodbye",
        "goodbyeya",
        "gtg",
        "gotta go",
        "got to go",
        "cya",
        "see you",
        "see ya later",
        "talk to you later",
        "gonna go know",
        "Till next time"],
        "responses": ["Goodbye!",
        "Alright, have a great time.",
        "Okay, You can close me.",
        "It would have been better if you were with me for a some more time. Anyways see you asap.']}
    ```
* `words` :\
    This is a set of words that chatbot's intents recognise.
* `classes` :\
    This is a list of classes (Intent names). We will use this to guess the output given from tensor-flow model.
* `docs` :\
    This will hold the dataset or training data.
* `ign_lets` :\
    This is a tuple of letter to ignore while we make our dataset.

In [2]:
stemmer                 = LancasterStemmer()
#stemmer.stem("saying") => "say"
intents : List[dict]    = load(open('./data/intents.json'))
words                   = set()
classes : List[str]     = []
docs : List[Tuple[List[str], str]] = []
ign_lets                = ('?','!',',','.')

### Creating the dataset
We will loop through all intents and for each intent we will loop through all patterns that belongs to the intent. We will tokenize and then stem the words in the patterns and then add them to `words` and `docs`. `words` is a set of stemmed words so we add the stemmed word to it and for `docs` variable, we make a list of stemmed words for each pattern and create a tuple out of it where first element of the tuple becomes the list of words and the second element is the `class` or say `intent name` that perticular pattern belongs to.

In [3]:
for intent in intents:
    for pattern in intent['patterns']:
        tokens = word_tokenize(pattern)
        stemmed_tokens = []
        for word in tokens:
            word = stemmer.stem(word.lower())
            if word not in ign_lets:
                words.add(word)
                stemmed_tokens.append(word)
        docs.append((stemmed_tokens, intent['intent']))
    if not intent['intent'] in classes:
        classes.append(intent['intent'])

### Preparing training data

Computer understands 0's and 1's easily so we convert words and classes to a bag of words and list containing 0 and 1.

In [24]:
classes   = sorted(list(classes))
words     = sorted(list(words))
training  = []
out_empty = [0] * len(classes)

for doc in docs:
    bag : List[int] = []
    for word in words:
        bag.append(1) if word in doc[0] else bag.append(0)

    out_row = out_empty[:] # Here we make a copy
    out_row[classes.index(doc[1])] = 1
    training.append((bag, out_row))

# Shuffling the training data
training = np.array(training, dtype=tuple)
np.random.shuffle(training)
train_x, train_y = list(training[:, 0]), list(training[:, 1])

In [25]:
len(train_x[0])

250

### Creating model
Creating a new Sequential model where we add layers and then feed the training data to the model and then train the model.

In [26]:
model = Sequential(
    layers = [
        Dense(125, 'relu', input_shape = (len(words),)),
        Dropout(0.5, input_shape = (len(words),)),
        Dense(86, 'relu', input_shape = (len(words),)),
        Dropout(0.5, input_shape = (len(words),)),
        Dense(len(classes), 'softmax')
    ],
    name = "Tensor-Bot"
)
model.summary()


Model: "Tensor-Bot"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 125)               31375     
_________________________________________________________________
dropout_6 (Dropout)          (None, 125)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 86)                10836     
_________________________________________________________________
dropout_7 (Dropout)          (None, 86)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 66)                5742      
Total params: 47,953
Trainable params: 47,953
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True),
    'categorical_crossentropy', ['accuracy']
)

hist = model.fit(train_x, train_y, epochs = 225, batch_size = 5, verbose = 0)
model.save('data/TensorBot_v2_.h5')

In [28]:
np.array(hist.history['accuracy'])

array([0.01687764, 0.04219409, 0.05063291, 0.06751055, 0.06329114,
       0.08860759, 0.08860759, 0.12658228, 0.15189873, 0.16455697,
       0.17299578, 0.24050634, 0.21097046, 0.23628692, 0.27004218,
       0.33333334, 0.33755276, 0.40928271, 0.40084389, 0.35021096,
       0.4135021 , 0.48101267, 0.50210971, 0.48945147, 0.52320677,
       0.54008436, 0.52742618, 0.55696201, 0.64135021, 0.60759491,
       0.5527426 , 0.62869197, 0.61181432, 0.70042193, 0.66244727,
       0.72151899, 0.70042193, 0.67510551, 0.70042193, 0.6919831 ,
       0.69620252, 0.70886075, 0.71308017, 0.75105482, 0.7763713 ,
       0.71729958, 0.75949365, 0.75105482, 0.78059071, 0.71729958,
       0.73839664, 0.80590719, 0.79746836, 0.78481013, 0.8101266 ,
       0.76793247, 0.8101266 , 0.81856543, 0.83544302, 0.84388185,
       0.83966243, 0.8101266 , 0.80590719, 0.84388185, 0.83122361,
       0.79746836, 0.83966243, 0.82700419, 0.82700419, 0.82700419,
       0.81856543, 0.84810126, 0.78481013, 0.85654008, 0.86075

### Saving some side data usefull for predicting the output
We will save `words` and `classes` in a pkl (pickle) file. This will prevent ous from recalculating the data when we predict.

In [16]:
pkl.dump((words, classes), open('data/chatbot_dump_v2_.pkl', 'wb'))